In [1]:
import xlrd
from os import path

In [2]:
arcpy.env.overwriteOutput = True

working = path.normpath('D:/Staging/acs2020/')
twb = path.join(working, 'Excel_Workbooks_for_Tracts')
bwb = path.join(working, 'Excel_Workbooks_for_BlockGroups')
tgdb = path.join(working, path.normpath('Geodatabase_Tables/Tables_for_Tracts.gdb'))
bgdb = path.join(working, path.normpath('Geodatabase_Tables/Tables_for_BlockGroups.gdb'))

exports = {twb: tgdb, bwb: bgdb}

for k in exports.keys():
    arcpy.env.workspace = k
    workbook_list = [i for i in arcpy.ListWorkspaces() if path.basename(i) != 'XLSX_format_tables']
    for workbook in workbook_list:
        desc = arcpy.Describe(workbook)
        workbook_name = desc.basename
        open_book = xlrd.open_workbook(workbook)
        workbook_sheets = open_book.sheet_names()
        for sheet in workbook_sheets:
            if sheet != 'Data_Dictionary':
                arcpy.ExcelToTable_conversion(workbook, path.join(exports[k], workbook_name), sheet)

In [4]:
arcpy.env.addOutputsToMap = False

from os import path
from datetime import datetime

start = datetime.now()

table_gdb = [
#     path.normpath('D:/Staging/acs2020/Geodatabase_Tables/Tables_for_Tracts.gdb'), 
    path.normpath('D:/Staging/acs2020/Geodatabase_Tables/Tables_for_BlockGroups.gdb')
]
features_gdb = [
#     path.normpath('D:/Staging/acs2020/Tracts_NM_ACS2020.gdb'), 
    path.normpath('D:/Staging/acs2020/BlockGroups_MRCOG_ACS2020.gdb')
]
fd = [
#     'NM_Tracts_2020', 
    'MRCOG_BlockGroups_2020'
]
fc = [
#     'NM_Tracts_2020', 
    'MRCOG_BlockGroups_2020'
]

fjoin = 'GEOID'
tjoin = 'GIS_Join_Match_Code'

for i in range(len(table_gdb)):
    print('Starting GDB {} of {}'.format(i+1, len(table_gdb)))
    arcpy.env.workspace = table_gdb[i]
    #arcpy.env.overwriteOutput = True
    features = path.join(features_gdb[i], fd[i], fc[i])
    table_list = [str(t) for t in arcpy.ListTables()]
    print('Found {} tables in GDB {}'.format(len(table_list), i+1))
    for t in table_list:
    	out_fc = path.join(features_gdb[i], t)
    	arcpy.CopyFeatures_management(features, out_fc)
    	join_fields = [str(f.name) for f in arcpy.ListFields(t)][2:]
    	arcpy.JoinField_management(out_fc, fjoin, t, tjoin, join_fields)
    print('Finished GDB {} of {}'.format(i+1, len(table_gdb)))

print (datetime.now() - start)

Starting GDB 1 of 1
Found 60 tables in GDB 1
Finished GDB 1 of 1
0:04:06.865109
